In [2]:
import pandas as pd
import os

## Tools

In [3]:
user_df = pd.read_json(os.path.join("data", "data_from_llm", "user.json"))
buy_history_df = pd.read_json(os.path.join("data", "data_from_llm", "buy_history.json"))
category_df = pd.read_json(os.path.join("data", "data_from_llm", "category.json"))
subcategory_df = pd.read_json(os.path.join("data", "data_from_llm", "subcategory.json"))
product_df = pd.read_json(os.path.join("data", "data_from_llm", "product.json"))
specification_df = pd.read_json(os.path.join(r"data", "data_from_llm", "specification.json"))
return_history_df = pd.read_json(os.path.join(r"data", "data_from_llm", "return_history.json"))
review_df = pd.read_json(os.path.join(r"data", "data_from_llm", "review.json"))

### Discovery Tool

In [4]:
# ------------------------------------------------------
# 1. Search Products
# ------------------------------------------------------
def search_products(category=None, subcategory=None, price_range=None, keywords=None, limit=50):
    """
    Search products by category, subcategory, price range, and optional keywords.
    """
    df = product_df.copy()

    # Subset early to avoid unnecessary joins
    if subcategory:
        sub_ids = subcategory_df.loc[subcategory_df['subcategory_name'].str.lower() == subcategory.lower(), 'subcategory_id']
        df = df[df['subcategory_id'].isin(sub_ids)]

    if category and not subcategory:
        cat_ids = category_df.loc[category_df['category_name'].str.lower() == category.lower(), 'category_id']
        sub_ids = subcategory_df.loc[subcategory_df['category_id'].isin(cat_ids), 'subcategory_id']
        df = df[df['subcategory_id'].isin(sub_ids)]

    if price_range:
        low, high = price_range
        df = df[(df['price'] >= low) & (df['price'] <= high)]

    if keywords:
        df = df[df['product_description'].str.contains(keywords, case=False, na=False)]

    return df.head(limit)


In [5]:
search_products(category="electronics", subcategory="laptop", price_range=(0, 2000), keywords="laptop", limit=50)

,product_id,subcategory_id,product_name,items_included,price,product_description,return_window
0,1001,101,ASUS ROG Strix G16 Gaming Laptop,"Laptop, Power Adapter, User Manual",1499.99,16-inch gaming laptop with Intel Core i9 proce...,30
1,1002,101,Apple MacBook Pro M2,"MacBook Pro, USB-C Power Adapter, USB-C Charge...",1799.99,"14-inch laptop with Apple M2 Pro chip, 16GB RA...",14
2,1003,101,Dell XPS 15 Ultrabook,"Laptop, Power Adapter, Quick Start Guide",1599.99,"15.6-inch laptop with Intel Core i7 processor,...",30
3,1004,101,Lenovo ThinkPad X1 Carbon,"Laptop, 65W AC Adapter, ThinkPad Pen",1399.99,14-inch business laptop with Intel Core i7 pro...,30
4,1005,101,HP Spectre x360 Convertible,"Laptop, Power Adapter, HP Pen, Protective Sleeve",1299.99,13.5-inch 2-in-1 laptop with Intel Core i7 pro...,30
5,1006,101,MSI Stealth 16 Studio,"Laptop, 240W Power Adapter, User Guide",1899.99,16-inch content creation laptop with Intel Cor...,30
6,1007,101,Acer Predator Helios 300,"Laptop, Power Adapter, User Manual",1199.99,15.6-inch gaming laptop with Intel Core i7 pro...,30
7,1008,101,Microsoft Surface Laptop 5,"Laptop, Power Supply, Quick Start Guide",1299.99,"13.5-inch laptop with Intel Core i7 processor,...",60
9,1010,101,LG Gram 17,"Laptop, Power Adapter, User Manual",1399.99,17-inch ultralight laptop with Intel Core i7 p...,30


In [6]:
# ------------------------------------------------------
# 2. Rank Products by Reviews
# ------------------------------------------------------
def rank_products_by_reviews(category=None, subcategory=None, limit=10):
    """
    Rank products by average rating from reviews.
    """
    reviews = review_df.groupby('product_id')['rating'].mean().reset_index(name='avg_rating')

    df = product_df.merge(reviews, on='product_id', how='inner')

    if subcategory:
        sub_ids = subcategory_df.loc[subcategory_df['subcategory_name'].str.lower() == subcategory.lower(), 'subcategory_id']
        df = df[df['subcategory_id'].isin(sub_ids)]

    if category and not subcategory:
        cat_ids = category_df.loc[category_df['category_name'].str.lower() == category.lower(), 'category_id']
        sub_ids = subcategory_df.loc[subcategory_df['category_id'].isin(cat_ids), 'subcategory_id']
        df = df[df['subcategory_id'].isin(sub_ids)]

    return df.sort_values('avg_rating', ascending=False).head(limit)

In [7]:
rank_products_by_reviews(subcategory="laptop", limit=5)

,product_id,subcategory_id,product_name,items_included,price,product_description,return_window,avg_rating
1,1002,101,Apple MacBook Pro M2,"MacBook Pro, USB-C Power Adapter, USB-C Charge...",1799.99,"14-inch laptop with Apple M2 Pro chip, 16GB RA...",14,4.6875
0,1001,101,ASUS ROG Strix G16 Gaming Laptop,"Laptop, Power Adapter, User Manual",1499.99,16-inch gaming laptop with Intel Core i9 proce...,30,4.3750


In [8]:
# ------------------------------------------------------
# 3. Get Product Reviews
# ------------------------------------------------------
def get_product_reviews(product_id=None, product_name=None, limit=20):
    """
    Retrieve reviews for a given product.
    """
    if product_id:
        return review_df[review_df['product_id'] == product_id].head(limit)

    if product_name:
        prod_ids = product_df.loc[product_df['product_name'].str.lower() == product_name.lower(), 'product_id']
        return review_df[review_df['product_id'].isin(prod_ids)].head(limit)

    return pd.DataFrame()  # Empty if no filter


In [9]:
get_product_reviews(product_name="Apple MacBook Pro M2", limit=10)

,review_id,user_id,product_id,rating,review_title,review_text,review_date,helpful_votes_count
16,R0017,17,1002,5,Perfect Balance of Power and Battery Life,The MacBook Pro M2 is everything Apple promise...,2025-06-25,156
17,R0018,18,1002,5,A Content Creator's Dream Machine,As a professional photographer and videographe...,2025-08-03,134
18,R0019,19,1002,4,"Nearly Perfect, Just Needs More Ports",The MacBook Pro M2 is an exceptional machine. ...,2025-05-14,98
19,R0020,20,1002,5,Worth Every Penny for Professionals,"After using Windows laptops for years, I switc...",2025-03-20,112
20,R0021,21,1002,4,"Excellent Performance, But Storage Fills Quickly",The MacBook Pro with M2 Pro is a powerhouse fo...,2025-02-05,87
21,R0022,22,1002,5,Unmatched Efficiency and Performance,"As a software developer, the MacBook Pro M2 ha...",2025-07-11,103
22,R0023,23,1002,5,The Ultimate Productivity Machine,This MacBook Pro has exceeded all my expectati...,2025-04-03,95
23,R0024,24,1002,4,Excellent Machine with Minor Software Limitations,The hardware of the MacBook Pro M2 is nearly p...,2025-01-30,78
24,R0025,25,1002,5,A Masterpiece of Engineering,"After six months with the MacBook Pro M2, I'm ...",2025-08-15,147
25,R0026,1,1002,5,Perfect Blend of Performance and Portability,The MacBook Pro with M2 Pro chip is the best l...,2025-06-07,89


In [10]:
# ------------------------------------------------------
# 4. Product Co-occurrence Analysis
# ------------------------------------------------------
def product_cooccurrence_analysis(product_id=None, subcategory=None, limit=10):
    """
    Find products frequently bought together with a given product or within a subcategory.
    """
    df = buy_history_df.copy()

    if product_id:
        orders = df[df['product_id'] == product_id]['order_id'].unique()
        cooccurring = df[df['order_id'].isin(orders) & (df['product_id'] != product_id)]
        result = cooccurring['product_id'].value_counts().head(limit).reset_index()
        result.columns = ['product_id', 'cooccurrence_count']
        return product_df.merge(result, on='product_id')

    if subcategory:
        sub_ids = subcategory_df.loc[subcategory_df['subcategory_name'].str.lower() == subcategory.lower(), 'subcategory_id']
        prod_ids = product_df.loc[product_df['subcategory_id'].isin(sub_ids), 'product_id']
        orders = df[df['product_id'].isin(prod_ids)]['order_id'].unique()
        cooccurring = df[df['order_id'].isin(orders) & (~df['product_id'].isin(prod_ids))]
        result = cooccurring['product_id'].value_counts().head(limit).reset_index()
        result.columns = ['product_id', 'cooccurrence_count']
        return product_df.merge(result, on='product_id')

    return pd.DataFrame()

In [11]:
product_cooccurrence_analysis(product_id=4050, limit=5)

,product_id,subcategory_id,product_name,items_included,price,product_description,return_window,cooccurrence_count


In [12]:
# ------------------------------------------------------
# 5. Correlation Purchase & Reviews
# ------------------------------------------------------
def correlation_purchase_reviews(category=None, subcategory=None, limit=10):
    """
    Identify products that are both frequently purchased and highly rated.
    """
    # Purchase frequency
    purchases = buy_history_df.groupby('product_id').size().reset_index(name='purchase_count')

    # Avg ratings
    reviews = review_df.groupby('product_id')['rating'].mean().reset_index(name='avg_rating')

    df = purchases.merge(reviews, on='product_id', how='inner')
    df = product_df.merge(df, on='product_id', how='inner')

    if subcategory:
        sub_ids = subcategory_df.loc[subcategory_df['subcategory_name'].str.lower() == subcategory.lower(), 'subcategory_id']
        df = df[df['subcategory_id'].isin(sub_ids)]

    if category and not subcategory:
        cat_ids = category_df.loc[category_df['category_name'].str.lower() == category.lower(), 'category_id']
        sub_ids = subcategory_df.loc[subcategory_df['category_id'].isin(cat_ids), 'subcategory_id']
        df = df[df['subcategory_id'].isin(sub_ids)]

    df['score'] = df['purchase_count'] * df['avg_rating']
    return df.sort_values('score', ascending=False).head(limit)


In [13]:
correlation_purchase_reviews(category="electronics", limit=10)

,product_id,subcategory_id,product_name,items_included,price,product_description,return_window,purchase_count,avg_rating,score
2,1011,102,Apple iPhone 15 Pro,"iPhone, USB-C Cable, Documentation",999.99,"6.1-inch smartphone with A17 Pro chip, 48MP ca...",14,16,4.750000,76.0
1,1002,101,Apple MacBook Pro M2,"MacBook Pro, USB-C Power Adapter, USB-C Charge...",1799.99,"14-inch laptop with Apple M2 Pro chip, 16GB RA...",14,16,4.687500,75.0
3,1012,102,Samsung Galaxy S24 Ultra,"Smartphone, USB-C Cable, Ejection Pin, Quick S...",1199.99,"6.8-inch smartphone with Snapdragon 8 Gen 3, 2...",30,15,4.733333,71.0
0,1001,101,ASUS ROG Strix G16 Gaming Laptop,"Laptop, Power Adapter, User Manual",1499.99,16-inch gaming laptop with Intel Core i9 proce...,30,16,4.375000,70.0
4,1021,103,Apple AirPods Pro 2,"AirPods Pro, MagSafe Charging Case, Silicone E...",249.99,Wireless earbuds with Active Noise Cancellatio...,14,12,4.750000,57.0
5,1022,103,Sony WF-1000XM5,"Earbuds, Charging Case, USB-C Cable, Hybrid Si...",279.99,Wireless earbuds with industry-leading noise c...,30,12,4.750000,57.0
8,1041,105,Sony Alpha a7 IV,"Camera Body, NP-FZ100 Rechargeable Battery, AC...",2499.99,33MP full-frame mirrorless camera with 4K 60p ...,30,12,4.750000,57.0
6,1031,104,Wacom Intuos Pro Large,"Graphic Tablet, Wacom Pro Pen 2, Pen Stand wit...",499.99,Professional graphic tablet with 8192 levels o...,30,10,4.700000,47.0
7,1032,104,XP-Pen Artist Pro 16,"Pen Display, Adjustable Stand, X3 Pro Stylus, ...",399.99,"16-inch drawing display with 2.5K resolution, ...",30,9,4.666667,42.0


In [14]:
# ------------------------------------------------------
# 6. Filter Products by Spec
# ------------------------------------------------------
def filter_products_by_spec(subcategory, specifications, limit=50):
    """
    Filter products by specification key-value pairs.
    specifications = dict, e.g. {"ram_gb": "16", "battery_hours": ">=8"}
    """
    # First subset by subcategory
    sub_ids = subcategory_df.loc[subcategory_df['subcategory_name'].str.lower() == subcategory.lower(), 'subcategory_id']
    df = product_df[product_df['subcategory_id'].isin(sub_ids)]

    # Join with specs only for relevant products
    specs = specification_df[specification_df['product_id'].isin(df['product_id'])]

    for key, value in specifications.items():
        if isinstance(value, str) and (value.startswith(">=") or value.startswith("<=")):
            op, num = value[:2], float(value[2:])
            if op == ">=":
                valid_ids = specs[(specs['spec_name'] == key) & (specs['spec_value'].astype(float) >= num)]['product_id']
            else:
                valid_ids = specs[(specs['spec_name'] == key) & (specs['spec_value'].astype(float) <= num)]['product_id']
        elif isinstance(value, str) and value.startswith("=="):
            num = float(value[2:])
            valid_ids = specs[(specs['spec_name'] == key) & (specs['spec_value'].astype(float) == num)]['product_id']
        else:  # direct match
            valid_ids = specs[(specs['spec_name'] == key) & (specs['spec_value'].astype(str).str.lower() == str(value).lower())]['product_id']

        df = df[df['product_id'].isin(valid_ids)]

    return df.head(limit)


In [15]:
# filter_products_by_spec(subcategory="laptop", specifications={"RAM": ">=16", "battery_hours": ">=8"})
# It should be a fuzzy match

In [16]:
specification_df[specification_df["product_id"] == 1001]

,spec_id,product_id,spec_name,spec_value
0,10001,1001,Processor,Intel Core i9-13980HX
1,10002,1001,RAM,32GB DDR5
2,10003,1001,Storage,1TB PCIe NVMe SSD
3,10004,1001,Graphics,NVIDIA GeForce RTX 4070 8GB GDDR6
4,10005,1001,Display,16-inch FHD+ 165Hz
5,10006,1001,Operating System,Windows 11 Home
6,10007,1001,Weight,2.5 kg


In [17]:
# specification_df["product_id"].value_counts()

In [18]:
# specification_df has two columns
list(specification_df["spec_name"].unique())

['Processor',
 'RAM',
 'Storage',
 'Graphics',
 'Display',
 'Operating System',
 'Weight',
 'Camera',
 'Battery',
 'Driver Size',
 'Connectivity',
 'Battery Life',
 'Noise Cancellation',
 'Water Resistance',
 'Charging',
 'Active Area',
 'Pressure Sensitivity',
 'Express Keys',
 'Multi-touch',
 'Pen',
 'Screen Size',
 'Resolution',
 'Color Gamut',
 'Shortcut Keys',
 'Tilt Support',
 'Sensor',
 'ISO Range',
 'Video',
 'Stabilization',
 'Autofocus',
 'Video Resolution',
 'Field of View',
 'Waterproof',
 'Photo Resolution',
 'Special Features',
 'Dynamic Range',
 'Recording Format',
 'Mount',
 'Thickness',
 'Material',
 'Dimensions',
 'Features',
 'Weight Range',
 'Adjustment Increments',
 'Dimensions (each)',
 'Handle Diameter',
 'Handle',
 'Color Coding',
 'Rack Dimensions',
 'Adjustment Mechanism',
 'Shape',
 'Weight Set',
 'Bottom',
 'Handle Length',
 'Rack',
 'Compatibility',
 'Grade',
 'Profile',
 'Edges',
 'Size',
 'Construction',
 'Intended Use',
 'Motor',
 'Speed Range',
 'Inclin

In [19]:
# Inner join on three tables: product, subcategory, and category
keep_list = ['category_name', 'subcategory_name', 'product_name', 'items_included', 'price', 'return_window', 'spec_name', 'spec_value']
result_df = category_df.merge(subcategory_df, on='category_id', how='inner') \
    .merge(product_df, on='subcategory_id', how='inner') \
    .merge(specification_df, on='product_id', how='inner')[keep_list]

# Display the result
result_df


,category_name,subcategory_name,product_name,items_included,price,return_window,spec_name,spec_value
0,Electronics,laptop,ASUS ROG Strix G16 Gaming Laptop,"Laptop, Power Adapter, User Manual",1499.99,30,Processor,Intel Core i9-13980HX
1,Electronics,laptop,ASUS ROG Strix G16 Gaming Laptop,"Laptop, Power Adapter, User Manual",1499.99,30,RAM,32GB DDR5
2,Electronics,laptop,ASUS ROG Strix G16 Gaming Laptop,"Laptop, Power Adapter, User Manual",1499.99,30,Storage,1TB PCIe NVMe SSD
3,Electronics,laptop,ASUS ROG Strix G16 Gaming Laptop,"Laptop, Power Adapter, User Manual",1499.99,30,Graphics,NVIDIA GeForce RTX 4070 8GB GDDR6
4,Electronics,laptop,ASUS ROG Strix G16 Gaming Laptop,"Laptop, Power Adapter, User Manual",1499.99,30,Display,16-inch FHD+ 165Hz
...,...,...,...,...,...,...,...,...
560,Sports,treadmill,Assault Fitness AssaultRunner Pro,"Treadmill, Assembly Tools, User Manual",2999.99,30,Motor,None (manual)
561,Sports,treadmill,Assault Fitness AssaultRunner Pro,"Treadmill, Assembly Tools, User Manual",2999.99,30,Speed Range,Unlimited (user-powered)
562,Sports,treadmill,Assault Fitness AssaultRunner Pro,"Treadmill, Assembly Tools, User Manual",2999.99,30,Incline,Fixed curved design
563,Sports,treadmill,Assault Fitness AssaultRunner Pro,"Treadmill, Assembly Tools, User Manual",2999.99,30,Running Surface,"17"" x 62"" (43.2 x 157.5 cm)"


In [32]:
result_df["category_name"].unique()

array(['Electronics', 'Sports'], dtype=object)

In [37]:
result_df["subcategory_name"].value_counts()

subcategory_name
smartphone        70
laptop            65
earphone          60
graphic tablet    60
camera            60
yoga mat          50
dumbbells         50
cricket bat       50
basketball        50
treadmill         50
Name: count, dtype: int64

In [33]:
result_df[result_df["subcategory_name"]=="treadmill"]["spec_name"].unique()

array(['Motor', 'Speed Range', 'Incline Range', 'Running Surface',
       'Display', 'Incline'], dtype=object)

In [34]:
result_df[result_df["subcategory_name"]=="earphone"]["spec_name"].unique()

array(['Driver Size', 'Connectivity', 'Battery Life',
       'Noise Cancellation', 'Water Resistance', 'Charging'], dtype=object)

In [23]:
result_df[(result_df["spec_name"] == "Graphics") & (result_df["subcategory_name"] == "laptop")][
    "spec_value"].value_counts()

spec_value
Intel Iris Xe Graphics               4
16-core GPU                          1
NVIDIA GeForce RTX 4070 8GB GDDR6    1
NVIDIA RTX 3050Ti 4GB GDDR6          1
NVIDIA RTX 4070 8GB GDDR6            1
NVIDIA RTX 3060 6GB GDDR6            1
NVIDIA RTX 4080 12GB GDDR6           1
Name: count, dtype: int64

In [24]:
result_df[(result_df["spec_name"] == "Weight Range") & (result_df["subcategory_name"] == "dumbbells")][
    "spec_value"].value_counts()


spec_value
5 to 52.5 lbs (2.3 to 23.8 kg) per dumbbell                     1
3 pairs: 3 lbs, 5 lbs, 8 lbs (1.4 kg, 2.3 kg, 3.6 kg)           1
10 to 55 lbs (4.5 to 25 kg)                                     1
5-50 lbs (2.3-22.7 kg) per hand, expandable to 90 lbs           1
11 to 71.5 lbs (5 to 32.4 kg)                                   1
5 pairs: 5, 10, 15, 20, 25 lbs (2.3, 4.5, 6.8, 9.1, 11.3 kg)    1
Name: count, dtype: int64

In [25]:
# subcategory_df

In [26]:
product_df[product_df["subcategory_id"]==101]

,product_id,subcategory_id,product_name,items_included,price,product_description,return_window
0,1001,101,ASUS ROG Strix G16 Gaming Laptop,"Laptop, Power Adapter, User Manual",1499.99,16-inch gaming laptop with Intel Core i9 proce...,30
1,1002,101,Apple MacBook Pro M2,"MacBook Pro, USB-C Power Adapter, USB-C Charge...",1799.99,"14-inch laptop with Apple M2 Pro chip, 16GB RA...",14
2,1003,101,Dell XPS 15 Ultrabook,"Laptop, Power Adapter, Quick Start Guide",1599.99,"15.6-inch laptop with Intel Core i7 processor,...",30
3,1004,101,Lenovo ThinkPad X1 Carbon,"Laptop, 65W AC Adapter, ThinkPad Pen",1399.99,14-inch business laptop with Intel Core i7 pro...,30
4,1005,101,HP Spectre x360 Convertible,"Laptop, Power Adapter, HP Pen, Protective Sleeve",1299.99,13.5-inch 2-in-1 laptop with Intel Core i7 pro...,30
5,1006,101,MSI Stealth 16 Studio,"Laptop, 240W Power Adapter, User Guide",1899.99,16-inch content creation laptop with Intel Cor...,30
6,1007,101,Acer Predator Helios 300,"Laptop, Power Adapter, User Manual",1199.99,15.6-inch gaming laptop with Intel Core i7 pro...,30
7,1008,101,Microsoft Surface Laptop 5,"Laptop, Power Supply, Quick Start Guide",1299.99,"13.5-inch laptop with Intel Core i7 processor,...",60
8,1009,101,Razer Blade 15 Advanced,"Laptop, Power Adapter, Razer Stickers",2499.99,15.6-inch premium gaming laptop with Intel Cor...,14
9,1010,101,LG Gram 17,"Laptop, Power Adapter, User Manual",1399.99,17-inch ultralight laptop with Intel Core i7 p...,30


In [27]:
result_df[result_df["subcategory_name"] == "dumbbells"].to_csv("result_df.csv")

In [28]:
pwd

'D:\\Study\\Programs\\TSAI\\EAG_V1_CAPESTONE\\dev_docs'